# PyPerceive to select and load Percept recordings 

## 0a. Loading default packages and functions

In [2]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 

from importlib import reload          


#### check package versions

developed with:
- Python sys 3.10.8
- pandas 1.5.1
- numpy 1.23.4
- mne_bids 0.11.1
- mne 1.2.3

In [3]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)


Python sys 3.10.8 | packaged by conda-forge | (main, Nov 24 2022, 14:07:00) [MSC v.1916 64 bit (AMD64)]
pandas 1.5.1
numpy 1.23.4
mne_bids 0.11.1
mne 1.2.3


## 0b. Loading pyPerceive functions

In [4]:
def add_and_set_code_folder_in_notebook():
    """
    while working in the local pyPerceive repo,
    find and set path to the PyPerceive code folder

    use function in notebook first, to locate the local
    repo and enable import of pyPerceive functions
    """
    project_path = os.getcwd()

    while project_path[-10:] != 'PyPerceive':
        project_path = os.path.dirname(project_path)

    code_path = os.path.join(project_path, 'code')
    sys.path.append(code_path)

    # change directory to code path
    os.chdir(code_path)
    
    return print(f'working dir set to: {code_path}')


In [5]:
## MAIN FUNCTION FOR DATA IMPORT

# change working directory to ensure correct loading of own functions
add_and_set_code_folder_in_notebook()

# import main class to work with
from PerceiveImport.classes import main_class


working dir set to: c:\Users\habetsj\Research\projects\PyPerceive\code


In [6]:
## IMPORT ALL SUB CLASSES AND FUNCTIONS FOR DEBUGGING
from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class, chronic_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

## 1. Test Data Loading for Streaming and Survey

In [7]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further

sub024 = main_class.PerceiveData(
    sub = "024", 
    incl_modalities=['survey', 'streaming', 'indefiniteStreaming'],
    incl_session = ["fu18m"],
    incl_condition =['m0s0'],
    incl_task = ["rest"],
    incl_contact = ["RingL", "SegmInterR", "SegmIntraR"],
    import_json=False,
    warn_for_metaNaNs=True,
    # use_bids=True,  # TODO: add to functionality
)



	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

In [17]:
sub024.streaming.fu18m.m0s0.rest.run1.data

<RawArray | 6 x 30187 (120.7 s), ~1.4 MB, data loaded>

In [ ]:
sub024.indefiniteStreaming.fu18m.m0s0.rest.run1.data.ch_names

In [ ]:
sub024.streaming.fu18m.m0s0.rest.run1.data

In [ ]:
sub024.streaming.fu18m.m0s0.rest.run1.data.get_data().shape

In [ ]:
print(sub024.indefiniteStreaming.fu18m.m0s0.rest.run1.data.get_data().shape)
print(sub024.indefiniteStreaming.fu18m.m0s0.rest.run1.data.ch_names)

In [ ]:
sub024.survey.fu18m.m0s0.rest.SegmInterR.run1.data.get_data()

## 2. Test Data Loading for Chronic

In [18]:
# in case of debugging
from PerceiveImport.methods import extract_chronic_timeline_samples as extract_chronic

In [19]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further
importlib.reload(main_class)
importlib.reload(modality_class)
importlib.reload(extract_chronic)

importlib.reload(chronic_class)
importlib.reload(extract_chronic)

dat = main_class.PerceiveData(
    sub = "015", 
    incl_modalities=['chronic'],
    import_json=True,
    warn_for_metaNaNs=False,
)



Add session 2021-01-11T07:28:47Z from Left hemisphere

Add session 2021-01-12T07:28:47Z from Left hemisphere

Add session 2021-01-11T07:28:47Z from Right hemisphere

Add session 2021-01-12T07:28:47Z from Right hemisphere

Add session 2021-01-19T07:28:47Z from Left hemisphere

Add session 2021-01-19T07:28:47Z from Right hemisphere

Add session 2022-01-25T07:28:47Z from Left hemisphere

Add session 2022-01-25T07:28:47Z from Right hemisphere

Add session 2021-01-15T07:28:47Z from Left hemisphere

Add session 2021-01-16T07:28:47Z from Left hemisphere

Add session 2021-01-17T07:28:47Z from Left hemisphere

Add session 2021-01-18T07:28:47Z from Left hemisphere

Add session 2021-01-15T07:28:47Z from Right hemisphere

Add session 2021-01-16T07:28:47Z from Right hemisphere

Add session 2021-01-17T07:28:47Z from Right hemisphere

Add session 2021-01-18T07:28:47Z from Right hemisphere

Add session 2021-01-21T07:28:47Z from Left hemisphere

Add session 2021-01-22T07:28:47Z from Left hemisphere

A

In [20]:
dat.chronic.data

,local_time,PSD_Left,freq_Left,contact_Left,group_name_Left,stim_amp_Left,PSD_Right,freq_Right,contact_Right,group_name_Right,stim_amp_Right
utc_time,,,,,,,,,,,
2021-01-11T22:55:48Z,2021-01-11 23:55:48 CET+0100,1644,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.1,374,21.48,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.1
2021-01-11T22:45:48Z,2021-01-11 23:45:48 CET+0100,6368,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.1,684,21.48,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.1
2021-01-11T22:35:48Z,2021-01-11 23:35:48 CET+0100,6803,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.1,867,21.48,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.1
2021-01-11T22:25:48Z,2021-01-11 23:25:48 CET+0100,6466,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.1,761,21.48,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.1
2021-01-11T22:15:48Z,2021-01-11 23:15:48 CET+0100,4703,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.1,713,21.48,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.1
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-13T23:46:24Z,2021-01-14 00:46:24 CET+0100,1447,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.4,150,23.44,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.4
2021-01-13T23:36:24Z,2021-01-14 00:36:24 CET+0100,223,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.4,74,23.44,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.4
2021-01-13T23:26:24Z,2021-01-14 00:26:24 CET+0100,435,20.51,SensingChannelDef.ZERO_TWO_LEFT,GroupIdDef.GROUP_A,1.4,79,23.44,SensingChannelDef.ZERO_TWO_RIGHT,GroupIdDef.GROUP_A,1.4


In [ ]:
dat.chronic.data

## 3. Direct access JSONs

In [22]:
json_fname = 'Report_Json_Session_Report_20210618T111250.json'
j = load_rawfile.load_sourceJSON('024', json_fname)

In [25]:
print(j.keys())

dict_keys(['AbnormalEnd', 'FullyReadForSession', 'FeatureInformationCode', 'SessionDate', 'SessionEndDate', 'ProgrammerTimezone', 'ProgrammerUtcOffset', 'ProgrammerLocale', 'ProgrammerVersion', 'PatientInformation', 'DeviceInformation', 'BatteryInformation', 'LeadConfiguration', 'Stimulation', 'Groups', 'BatteryReminder', 'MostRecentInSessionSignalCheck', 'Impedance', 'GroupHistory', 'LfpMontageTimeDomain', 'BrainSenseTimeDomain', 'BrainSenseLfp', 'LFPMontage', 'DiagnosticData'])


In [28]:
prc_data_codes = {
    'signal_test': 'CalibrationTests',
    'streaming': 'BrainSenseTimeDomain',
    'survey': 'LfpMontageTimeDomain',
    'indef_streaming': 'IndefiniteStreaming'
}

In [34]:
mod = 'streaming'

list_of_streamings = j[prc_data_codes[mod]]
n_streamings = len(list_of_streamings)

# n_exp_streamings = extract from metadata
# check whether n-streamings match metdata table 
# if n_streamings == n_streamings: ...

for dat in list_of_streamings:
    print(dat)

6

In [38]:
print(
    len(list_of_streamings[0]['GlobalPacketSizes']),
    len(list_of_streamings[0]['TicksInMses']),
    len(list_of_streamings[0]['GlobalSequences'])
    )

1467 3912 1693


In [79]:
1693 - 1467

226

In [40]:
dat = list_of_streamings[0]

In [73]:
print(dat.keys())

sfreq = dat['SampleRateInHz']
starttime = dat['FirstPacketDateTime']
ch_name = dat['Channel']



dict_keys(['Pass', 'GlobalSequences', 'GlobalPacketSizes', 'TicksInMses', 'Channel', 'Gain', 'FirstPacketDateTime', 'SampleRateInHz', 'TimeDomainData'])


'ZERO_TWO_LEFT'

In [74]:
def convert_list_string_floats(
    string_list
):
    try:
        floats = [float(v) for v in string_list.split(',')]
    except:
        floats = [float(v) for v in string_list[:-1].split(',')]

    return floats


In [78]:
gs_floats = convert_list_string_floats(dat['GlobalSequences'])

diffs = np.diff(gs_floats) == 2

sum(diffs)

122

In [64]:
len(dat['TimeDomainData']) 

30563